In [7]:
!head SMSSpamCollection.csv

ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
ham,Ok lar... Joking wif u oni...
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham,U dun say so early hor... U c already then say...
ham,"Nah I don't think he goes to usf, he lives around here though"
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"
ham,Even my brother is not like to speak with me. They treat me like aids patent.
ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load dataset
# The SMSCorpus.csv file is tab-separated and has no header.
# The first column is the label (ham or spam) and the second column is the message.
# It appears the separator might not be a simple tab, or there are inconsistencies.
# Let's try reading with comma separation and then splitting the columns.
data = pd.read_csv("SMSSpamCollection.csv", sep=",", header=None, names=["label_message"])

# Split the combined column into 'label' and 'message'
data['label'] = data['label_message'].apply(lambda x: x.split(',', 1)[0])
data['message'] = data['label_message'].apply(lambda x: x.split(',', 1)[1] if ',' in x else '')

# Drop the original combined column
data = data.drop(columns=['label_message'])

print(data.head())

# Encode labels (ham=0, spam=1)
le = LabelEncoder()
data["label_num"] = le.fit_transform(data["label"])

# Features (text) and target (label)
X = data["message"]
y = data["label_num"]

# Convert text into numerical features (TF-IDF)
vectorizer = TfidfVectorizer(stop_words="english")
X_tfidf = vectorizer.fit_transform(X)

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=2)

# Decision Tree Classifier
dt = DecisionTreeClassifier(criterion="entropy")
dt.fit(x_train, y_train)

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)

# Predictions
y_dt_pred = dt.predict(x_test)
y_lr_pred = lr.predict(x_test)

print("y_test shape:", y_test.shape)
print("y_dt_pred shape:", y_dt_pred.shape)
print("y_lr_pred shape:", y_lr_pred.shape)

# Accuracy
accuracy_dt = accuracy_score(y_test, y_dt_pred)
accuracy_lr = accuracy_score(y_test, y_lr_pred)

print("Decision Tree Accuracy:", accuracy_dt)
print("Logistic Regression Accuracy:", accuracy_lr)

# Example new prediction
sample = ["Congratulations! You've won a free ticket to Bahamas. Call now!"]
sample_tfidf = vectorizer.transform(sample)
print("Prediction (0=ham, 1=spam):", lr.predict(sample_tfidf))

                                                  label  \
ham                               Go until jurong point   
ham                       Ok lar... Joking wif u oni...   
spam  Free entry in 2 a wkly comp to win FA Cup fina...   
ham   U dun say so early hor... U c already then say...   
ham                    Nah I don't think he goes to usf   

                                                message  
ham    crazy.. Available only in bugis n great world...  
ham                                                      
spam                                                     
ham                                                      
ham                         he lives around here though  
y_test shape: (1115,)
y_dt_pred shape: (1115,)
y_lr_pred shape: (1115,)
Decision Tree Accuracy: 0.03766816143497758
Logistic Regression Accuracy: 0.014349775784753363
Prediction (0=ham, 1=spam): [3521]


In [11]:
# Build contingency table
table = [[0, 0], [0, 0]]

for i in range(len(y_test)):
    if y_test.iloc[i] == y_lr_pred[i] and y_test.iloc[i] != y_dt_pred[i]:
        table[0][1] += 1  # Logistic Regression correct, Decision Tree wrong
    elif y_test.iloc[i] != y_lr_pred[i] and y_test.iloc[i] == y_dt_pred[i]:
        table[1][0] += 1  # Logistic Regression wrong, Decision Tree correct
    elif y_test.iloc[i] == y_lr_pred[i] and y_test.iloc[i] == y_dt_pred[i]:
        table[0][0] += 1  # Both correct
    elif y_test.iloc[i] != y_lr_pred[i] and y_test.iloc[i] != y_dt_pred[i]:
        table[1][1] += 1  # Both wrong

# Print the contingency table
print("Contingency Table (Logistic Regression vs Decision Tree):")
print(pd.DataFrame(table,
                   index=["LogReg Correct", "LogReg Wrong"],
                   columns=["DTree Correct", "DTree Wrong"]))

# Perform McNemar's test
from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(table, exact=True)

# Summarize the finding
print("\nMcNemar's test statistic:", result.statistic)
print("p-value:", result.pvalue)

if result.pvalue < 0.05:
    print("Significant difference between the models.")
else:
    print("No significant difference between the models.")

Contingency Table (Logistic Regression vs Decision Tree):
                DTree Correct  DTree Wrong
LogReg Correct             13            3
LogReg Wrong               29         1070

McNemar's test statistic: 3.0
p-value: 2.5560148060321808e-06
Significant difference between the models.
